# **OPTICAL MODULE HEAT REMOVAL ANALYSIS - HEAT SINK DESIGN**

### This document covers the heat transfer analysis required to keep the optical module at steady state while operating at the max operating temperature of 70 degrees Celsius, and removing 2W. Follow along and find additional context in "Front Enclosure Analysis.pdf"

## **Conduction**

In [12]:
# Area and heat flux calculations for one surface
pad_area = (20.54*15.37)/1000**2  # m^2
heat = 1  # W
heat_flux = heat/pad_area  # W/m^2

In [13]:
# Material properties
Ka = 237  # W/mK for Aluminum
Kc = 401  # W/mK for Copper
Th = 70  # deg
Tc = 30  # deg

In [14]:
# Function to calculate required thickness to keep the device at steady state for conduction
def conduction_thickness(Ka, Kc, flux, Th, Tc):
    dT = Tc - Th
    La = -Ka * dT / flux
    Lc = -Kc * dT / flux
    return La.__round__(3), Lc.__round__(3)

L_aluminum, L_copper = conduction_thickness(Ka, Kc, heat_flux, Th, Tc)
print("Required thickness for Aluminium: ", L_aluminum, "m")
print("Required thickness for Copper: ", L_copper, "m")

Required thickness for Aluminium:  2.993 m
Required thickness for Copper:  5.064 m


### Copper conducts heat internally very easily, so it produces a small internal temperature gradient. Aluminum conducts less well, so it produces a larger internal temperature gradient. Therefore, for the same length and heat input, the reverse side of the aluminum block will still be cooler than the reverse side of the copper block.

### A slow removal of the heat is acceptable, seeing as the buildup of heat (to the point where it can be felt against the forehead) is slow. Aluminum is therefore acceptable. Aluminum is preferable due to its lower cost and weight, and the steeper temperature gradient means that the outer surface will be cooler to the touch while the system is kept at steady state, which is ergonomic. However, from the resulting length of metal required above, we see that conduction will not remove heat quickly or sufficiently.

## **Convection**
### But with convection, $q = hA(T_s - T_{inf})$, and we cannot find $T_s$ from conduction unless we complete some iterative calculations or 2D modelling. We will therefore approach this problem by using a thermal circuit and assuming the coefficient of convective heat transfer. The (limiting) length of aluminum we get from our 3D CAD model, and we iterate this as well as the area of convective heat transfer at the outer surface of the enclosure.

In [15]:
# Convection and thermal resistance calculations
L = 1/1000  # 1mm to metres
h = 15  # 15 W/m^2K For the size of my 323 sqft apartment room at 25degC and moderate forced convection (A/C blowing 25degC air around the room)
conv_area = 20*8.5/1000**2  # 20mm by 8.5mm at best
# currently 15mm by 8mm area for convection in the design


# Note the three parameters defined above are all variables that can be adjusted to achieve the desired cooling, 
# as they can be iterated on and adjusted in the 3D design.

R_cond = L / (Ka * pad_area)  # K/W
R_conv = 1 / (h * conv_area)  # K/W
R_total = R_cond + R_conv  # K/W
T_inf = 25  # degC

if R_total <= (Th - T_inf) / heat:
    print("Design is adequate to maintain at or cool the device below ", Th, "degC")
    print("Max thermal resistance allowed: ", round((Th - T_inf) / heat, 3), "K/W")
    print("Total thermal resistance of design: ", round(R_total, 3), "K/W")
else:
    print("Design is NOT adequate to maintain at or cool the device below ", Th, "degC")
    print("Maximum thermal resistance allowed: ", round((Th - T_inf) / heat, 3), "K/W")
    print("Total thermal resistance of design: ", round(R_total, 3), "K/W")
    print("Convective thermal resistance: ", round(R_conv, 3), "K/W")
    print("Conductive thermal resistance: ", round(R_cond, 3), "K/W")

Design is NOT adequate to maintain at or cool the device below  70 degC
Maximum thermal resistance allowed:  45.0 K/W
Total thermal resistance of design:  392.17 K/W
Convective thermal resistance:  392.157 K/W
Conductive thermal resistance:  0.013 K/W


### Note that the heat across each section of the thermal circuit is equal at steady state. Therefore, the equation:
- $q = h A_{conv}(T_h - T_{\infty})$ must hold

given that all parameters except the coefficient of heat transfer $h$ are known and constrained, it is the variable of suspect for why the equation does not hold and the left hand side is not equal to 1W (the design is not adequate)

In [16]:
h_opt = heat/(conv_area*(Th-T_inf))
print("A heat transfer coefficient of", round(h_opt,3),"W/m^2K or higher is required for the design to be adequate.")

A heat transfer coefficient of 130.719 W/m^2K or higher is required for the design to be adequate.


### This theory can be tested by using the found optimum coefficient to compute the thermal resistance that would make the design adequate or make the following equation hold:

-   $R_{cond} + R_{conv} \leq \Delta T/q$

In [17]:
opt_R_conv = 1 / (h_opt * conv_area)  # K/W
opt_R_total = R_cond + opt_R_conv  # K/W
print("With the optimum heat transfer coefficient, the total thermal resistance would be:", round(opt_R_total,3),"K/W")
print("Maximum thermal resistance allowed: ", round((Th - T_inf) / heat, 3), "K/W")

With the optimum heat transfer coefficient, the total thermal resistance would be: 45.013 K/W
Maximum thermal resistance allowed:  45.0 K/W


### The high 130 W/m^2K coefficient of heat transfer required at the outer face of the enclosure far exceeds the average value in a room with natural convection, and can only be achieved through very high velocity localised airflow. Seeing that this is the least practical solution as the ideal operating environment and conditions for the headset are centered around average, to make the thermal resistance equation hold we can also inspect the convective area.

In [18]:
poss_conv_area = heat/(h*(Th - T_inf))
print("With the current heat transfer coefficient of",round(h,3),"W/m^2K, " \
"the convection area required is:",round(poss_conv_area*1000**2,3),"cm^2")

With the current heat transfer coefficient of 15 W/m^2K, the convection area required is: 1481.481 cm^2


This is equivalent to a 380mm by 380mm square. The entire side cross section of the base enclosure is 65mm by 65mm.

### There are no practical, passive (heat sink), natural convection solutions to achieve the required heat extraction and dumping in this design.

# **OPTICAL MODULE HEAT REMOVAL ANALYSIS - DIRECT EXPOSURE**

### Direct exposure here is intended to mean that the volume available to fit the conceptual heat sink will be excavated entirely of material such that the heat dissipation pads of the optical module are exposed directly to ambient air. The design is analysed as a horizontal hotplate-cold plate heat transfer problem.

$q = hA_{pad_{-}area}(T_s - T_{inf})$

Continuing the analysis using one heat dissipation pad.

In [19]:
h_gap = heat/(pad_area*(Th - T_inf)) # W/m^2K
print("To use the entire pad area for convection, a heat transfer coefficient of",round(h_gap,3),"W/m^2K is required.")

To use the entire pad area for convection, a heat transfer coefficient of 70.39 W/m^2K is required.


In [20]:
R_rem = 1/(h*pad_area)  # K/W
print("With the current heat transfer coefficient of",round(h,3),"W/m^2K and the entire pad area used for convection, the total thermal resistance would be",round(R_rem,3),"K/W")

With the current heat transfer coefficient of 15 W/m^2K and the entire pad area used for convection, the total thermal resistance would be 211.171 K/W


In [21]:
print("The total thermal resistance of the previous design is:", round(R_total,3),"K/W")

The total thermal resistance of the previous design is: 392.17 K/W


### Exposing the optical module's thermal dissipation pads directly to ambient air has a lower (by about 180K) resistance to the transfer of 1W. The 1W will be dumped into the ambient air at lower temperature differences between $T_{h}$ and $T_{C}$ in the circuit. This design is preferable to the heat sink design and will be implemented.

### Heat transfer behaviour in both designs will be simulated and the temperature in the forehead region of the enclosure after 2 full hours of sustained use due to heat transfer from the optical module through the plastic enclosure will be used to determine whether or not an active cooling (forced convection) solution will be implemented.